Notebook to experiment with the lifespan estimator and assist in its development.

In [1]:
from quantumthreattracker.lifespan_estimator.lifespan_estimator import (
    HardwareRoadmap,
    LifespanEstimator,
)

In [2]:
hardware_roadmap = HardwareRoadmap()

In [3]:
hardware_roadmap.add(year=2040, qubit_type = "superconducting", n_qubits = 1e6, gate_error_rate = 1e-3)
hardware_roadmap.add(year=2045, qubit_type = "superconducting", n_qubits = 1e7, gate_error_rate = 1e-3)
hardware_roadmap.add(year=2050, qubit_type = "superconducting", n_qubits = 1e8, gate_error_rate = 1e-3)

hardware_roadmap.add(year=2040, qubit_type = "trapped ions", n_qubits = 1e5, gate_error_rate = 1e-6)
hardware_roadmap.add(year=2045, qubit_type = "trapped ions", n_qubits = 1e6, gate_error_rate = 1e-6)
hardware_roadmap.add(year=2050, qubit_type = "trapped ions", n_qubits = 1e7, gate_error_rate = 1e-6)

In [4]:
hardware_roadmap.list()

[{'timestamp': 2208988800.0,
  'hardware': [{'qubit_type': 'superconducting',
    'n_qubits': 1000000.0,
    'gate_error_rate': 0.001},
   [{'qubit_type': 'trapped ions',
     'n_qubits': 100000.0,
     'gate_error_rate': 1e-06}]]},
 {'timestamp': 2366841600.0,
  'hardware': [{'qubit_type': 'superconducting',
    'n_qubits': 10000000.0,
    'gate_error_rate': 0.001},
   [{'qubit_type': 'trapped ions',
     'n_qubits': 1000000.0,
     'gate_error_rate': 1e-06}]]},
 {'timestamp': 2524608000.0,
  'hardware': [{'qubit_type': 'superconducting',
    'n_qubits': 100000000.0,
    'gate_error_rate': 0.001},
   [{'qubit_type': 'trapped ions',
     'n_qubits': 10000000.0,
     'gate_error_rate': 1e-06}]]}]

In [5]:
lifespan_estimator = LifespanEstimator(hardware_roadmap)
timestamp = lifespan_estimator.estimate_lifespan("RSA", 8, formatted=True)

print(timestamp)

2040-01-01
